#### 5. Система бронирования авиакомпаний/отелей. Создайте систему бронирования, которая бронирует места в самолетах или номера в отелях. Он взимает различные тарифы за определенные участки самолета или отеля. Например, первый класс будет стоить больше, чем эконом. В гостиничных номерах есть пентхаусы, которые стоят дороже. Следите за тем, когда комнаты будут доступны и могут быть запланированы.

In [69]:
import datetime
from enum import Enum


class RoomClass(Enum):  # класс гостиницы
    Standart = "standart"
    Penthouse = "penthouse"

    def __repr__(self):
        return self.value


class TicketClass(Enum):  # класс авиабилета
    Econom = "econom"
    Business = "business"

    def __repr__(self):
        return self.value


RoomReservation = dict[str: int]  # { iso_date: booked_spaces_count }
HotelBookings = dict[RoomClass, RoomReservation]

# hotel_bookings: HotelBooking = {
#     RoomClass.Penthouse: {
#         "24.10.2022": 1,
#         "28.10.2022": 5
#     }
# }


class Hotel:
    def __init__(self, title: str, rooms: dict[RoomClass, int]):
        self.title = title
        self.rooms = {RoomClass.Standart: 0, RoomClass.Penthouse: 0, **rooms}

    def __repr__(self):
        return f'Hotel {self.title}, Vacant spaces: {self.rooms}'


class Airline:
    def __init__(self, title: str, tickets: tuple[int, int]):
        self.title = title
        self.tickets = tickets

    def __repr__(self):
        return f'Airline \'{self.title}\''


class Supervisor:
    def __init__(self, hotels: list[Hotel] = [], airlines: list[Airline] = []):
        self._hotels = hotels
        self._airlines = airlines

        self._hotel_bookings: HotelBookings = {}

    def add_hotel(self, hotel: Hotel):
        if hotel in self._hotels:
            return print(f'Oops \'{hotel.title}\' already on the list.')
        self._hotels += [hotel]

    def add_airline(self, airline: Airline):
        if airline in self._airlines:
            return print(f'Oops \'{airline.title}\' already on the list.')
        self._airlines += [airline]

    def remove_hotel(self, hotel_title: str):
        hotel_to_remove = self._find_hotel(hotel_title)

        if hotel_to_remove is None:
            return print(f'There\'s no hotel {hotel_title} on the list.')

        self._hotels.remove(hotel_to_remove)
        print(f'Removed hotel {hotel_title}')

    def remove_airline(self, airline_title: str):
        airline_to_remove = self._find_airline(airline_title)

        if airline_to_remove is None:
            return print(f'There\'s no airline {airline_title} on the list.')

        self._airlines.remove(airline_to_remove)
        print(f'Removed airline {airline_title}')

    def get_hotels(self):
        return self._hotels

    def get_airlines(self):
        return self._airlines

    def book_a_room(self, hotel_title: str, room_class: RoomClass, date: str):
        hotel_to_book = self._find_hotel(hotel_title)
        if hotel_to_book is None:
            return print(f'There\'s no hotel {hotel_title} on the list.')
        if self._has_vacant_rooms(hotel_to_book, room_class):
            hotel_to_book.rooms[room_class] -= 1
            self._create_room_reservation(room_class, date)
            print(
                f'Thank you! You have successfully booked a room in hotel \'{hotel_title}\'.')
        else:
            print(
                f'Sorry, there\'s no vacant spaces in {hotel_title}. Try another room class.')

    def book_a_plane_ticket(self, airline_title: str, ticket_class: TicketClass):
        pass

    def _create_room_reservation(self, room_class: RoomClass, date: str) -> RoomReservation:
        current_bookings = self._hotel_bookings
        # если еще не было брони текущего класса, создадим её и заполним пустыми данными
        if not (room_class in current_bookings.keys()):
            current_bookings = {**current_bookings, room_class: {}}

        # если бронь на текущую дату существует, увеличим кол-во забронированных комнат
        if date in current_bookings[room_class].keys():
            current_bookings[room_class][date] += 1
        else:  # иначе сохраним все предыдущие брони и добавим новую на новую дату
            current_bookings[room_class] = {
                **current_bookings[room_class], date: 1}

        self._hotel_bookings = current_bookings

    def _find_hotel(self, hotel_title: str) -> Hotel | None:
        return next(
            (hotel for hotel in self._hotels if hotel.title == hotel_title), None)

    def _find_airline(self, airline_title: str) -> Airline | None:
        return next(
            (airline for airline in self._airlines if airline.title == airline_title), None)

    def _has_vacant_rooms(self, hotel: Hotel, room_class: RoomClass) -> bool:
        return hotel.rooms[room_class] > 0


mariott = Hotel("Mariott", {RoomClass.Standart: 30, RoomClass.Penthouse: 10})
s7 = Airline("S7", (100, 30))

sv = Supervisor([mariott], [s7])

sv.book_a_room(mariott.title, RoomClass.Standart, "24.10.2022")
sv.book_a_room(mariott.title, RoomClass.Standart, "24.10.2022")
sv.book_a_room(mariott.title, RoomClass.Standart, "28.10.2022")
sv.book_a_room(mariott.title, RoomClass.Standart, "28.10.2022")
sv.book_a_room(mariott.title, RoomClass.Standart, "28.10.2022")

sv.book_a_room(mariott.title, RoomClass.Penthouse, "28.10.2022")


print(sv._hotel_bookings)


Thank you! You have successfully booked a room in hotel 'Mariott'.
Thank you! You have successfully booked a room in hotel 'Mariott'.
Thank you! You have successfully booked a room in hotel 'Mariott'.
Thank you! You have successfully booked a room in hotel 'Mariott'.
Thank you! You have successfully booked a room in hotel 'Mariott'.
Thank you! You have successfully booked a room in hotel 'Mariott'.
{standart: {'24.10.2022': 2, '28.10.2022': 3}, penthouse: {'28.10.2022': 1}}
